#### NAME: MURK ASAD (101424114)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train=pd.read_csv("https://raw.githubusercontent.com/subashgandyer/datasets/main/loan_train.csv")

In [3]:
test=pd.read_csv("https://raw.githubusercontent.com/subashgandyer/datasets/main/loan_test.csv")
#not using test dataset,as it has no labels

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [5]:
train.shape

(614, 13)

In [6]:
train.head(3)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y


## Data Cleaning

In [7]:
#Cleaning trainset

In [8]:
#drop loan_id
train.drop(columns='Loan_ID', axis=1, inplace=True)

In [9]:
train.head(2)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N


In [10]:
#drop loan_id
test.drop(columns='Loan_ID', axis=1, inplace=True)

In [11]:
#X = train.drop(columns='Loan_Status', axis=1)
#y = train['Loan_Status']

In [12]:
train.isna().sum()

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [13]:
#finding neumeric and categorical data
numeric_data = train.select_dtypes(include=[np.number]).columns
categorical_data = train.select_dtypes(exclude=[np.number]).columns
print('Numeric columns:', numeric_data)
print('Cat columns: ', categorical_data)

Numeric columns: Index(['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History'],
      dtype='object')
Cat columns:  Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'Property_Area', 'Loan_Status'],
      dtype='object')


### NUMERIC DATA

#### Filling NAN for numeric data: SimpleImputer

In [14]:
from sklearn.impute import SimpleImputer
imp=SimpleImputer(missing_values=np.nan, strategy='median')
train_num=imp.fit_transform(train[numeric_data])

In [15]:
np.isnan(train_num).sum()

0

In [16]:
train_num=pd.DataFrame(train_num)

In [17]:
train_num.isna().sum()

0    0
1    0
2    0
3    0
4    0
dtype: int64

In [18]:
train_num.head()

,0,1,2,3,4
0,5849.0,0.0,128.0,360.0,1.0
1,4583.0,1508.0,128.0,360.0,1.0
2,3000.0,0.0,66.0,360.0,1.0
3,2583.0,2358.0,120.0,360.0,1.0
4,6000.0,0.0,141.0,360.0,1.0


#### MinMaxScaler

In [19]:
#applicant and coapplicant income need scaling because the range is so high as compared to other columns
from sklearn.preprocessing import MinMaxScaler
minmax=MinMaxScaler()
train_num=minmax.fit_transform(train_num)

In [20]:
train_num=pd.DataFrame(train_num)

In [21]:
num_names=train[numeric_data].columns.tolist()
train_num.rename(columns=dict(zip(train_num.columns,num_names)), inplace=True)

In [22]:
train_num.head(3)

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
0,0.070489,0.000000,0.172214,0.74359,1.0
1,0.054830,0.036192,0.172214,0.74359,1.0
2,0.035250,0.000000,0.082489,0.74359,1.0


### CATEGORICAL DATA

#### Filling NAN for Categorical data

In [23]:
#using bfill and ffill for categorical columns
for col_name in categorical_data:
    train[col_name] = train[col_name].ffill().bfill()

In [24]:
train_cat=train[categorical_data]

In [25]:
train_cat.isna().sum()

Gender           0
Married          0
Dependents       0
Education        0
Self_Employed    0
Property_Area    0
Loan_Status      0
dtype: int64

In [26]:
train_cat.head()

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,Urban,Y
1,Male,Yes,1,Graduate,No,Rural,N
2,Male,Yes,0,Graduate,Yes,Urban,Y
3,Male,Yes,0,Not Graduate,No,Urban,Y
4,Male,No,0,Graduate,No,Urban,Y


#### Label Encoding

In [27]:
#applying label encoder for categorical
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
for col_name in categorical_data:
    train_cat[col_name] = label_encoder.fit_transform(train_cat[col_name])

<ipython-input-27-c57cbfe192c0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_cat[col_name] = label_encoder.fit_transform(train_cat[col_name])


In [28]:
cat_names=train[categorical_data].columns.tolist()
train_cat.rename(columns=dict(zip(train_cat.columns,cat_names)), inplace=True)

D:\Anaconda\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [29]:
train_cat.head(3)

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status
0,1,0,0,0,0,2,1
1,1,1,1,0,0,0,0
2,1,1,0,0,1,2,1


#### Combining both num and cat columns in a new dataframe

In [30]:
train_transform=pd.concat([train_num,train_cat], axis=1, join='inner')

In [31]:
train_transform.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status
0,0.070489,0.000000,0.172214,0.74359,1.0,1,0,0,0,0,2,1
1,0.054830,0.036192,0.172214,0.74359,1.0,1,1,1,0,0,0,0
2,0.035250,0.000000,0.082489,0.74359,1.0,1,1,0,0,1,2,1
3,0.030093,0.056592,0.160637,0.74359,1.0,1,1,0,1,0,2,1
4,0.072356,0.000000,0.191027,0.74359,1.0,1,0,0,0,0,2,1


In [32]:
train_transform.shape

(614, 12)

### Split dataset to X and y

In [33]:
X = train_transform.drop(columns='Loan_Status', axis=1)
y = train_transform['Loan_Status']

## Imbalanced Dataset

In [34]:
y.value_counts()

1    422
0    192
Name: Loan_Status, dtype: int64

#### Repeated StratifiedKFold split

In [35]:
from sklearn.model_selection import RepeatedStratifiedKFold

In [36]:
rskf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
for train_index, test_index in rskf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  27  28  29  30  32  33  35  36  37  38  39
  40  41  42  43  44  45  46  47  48  49  50  51  52  53  55  56  57  58
  60  61  62  63  64  65  66  67  68  69  70  72  73  74  75  76  77  78
  79  80  81  82  83  85  86  89  90  91  92  93  94  95  96  97  99 100
 101 102 104 105 106 107 108 109 110 111 112 114 115 116 117 118 119 120
 121 124 125 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141
 143 144 145 146 147 148 151 153 154 155 158 159 160 161 162 163 164 165
 166 167 169 170 171 172 173 175 177 179 180 181 182 183 184 185 186 187
 188 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206
 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224
 225 227 228 229 230 231 232 233 234 235 236 238 239 240 241 242 243 244
 245 246 247 248 250 251 252 253 255 256 257 258 259 260 263 264 265 266
 267 268 269 270 271 272 273 274 275 276 277

## Applying models

In [37]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
import timeit

### Without GridSearchCV

In [44]:
basicresults=pd.DataFrame(columns=['Algorithm','Accuracy'])#results from applying algorithms without grid search

params_list=[DecisionTreeClassifier(), 
            KNeighborsClassifier(), 
            LogisticRegression(),
            SVC(),
            RandomForestClassifier(),
            SGDClassifier(),
            ]

for model in params_list:
    model.fit(X_train, y_train)
    y_predict=model.predict(X_test)
    listofalgos=[model,round(accuracy_score(y_test,y_predict),4)]
    temp = pd.DataFrame([listofalgos], columns=['Algorithm','Accuracy'])
    basicresults=pd.concat([temp, basicresults], axis=0)

In [45]:
basicresults.reset_index(inplace=True)

In [48]:
basicresults.drop(columns=['index'], inplace=True)

In [49]:
basicresults.sort_values(by=['Accuracy'], ascending=False)

,Algorithm,Accuracy
0,SGDClassifier(),0.8525
2,SVC(),0.8525
3,LogisticRegression(),0.8525
1,"(DecisionTreeClassifier(max_features='auto', r...",0.8361
4,KNeighborsClassifier(),0.8033
5,DecisionTreeClassifier(),0.6557


### GridSearchCV

In [50]:
import warnings
warnings.filterwarnings('ignore')

In [51]:
from sklearn.model_selection import GridSearchCV

In [52]:
dt_list= {'criterion':['gini', 'entropy'], 'splitter':['best', 'random'], 'max_features':['auto', 'sqrt']}
knn_list={'n_neighbors':range(2,5), 'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute']}
log_list={'penalty':['l1','l2',None], 'solver':['liblinear','lbfgs']}
svm_list={'kernel':['linear', 'poly' 'rbf'],'gamma':['scale', 'auto']}
rf_list={'n_estimators': range(50,100),'max_leaf_nodes': range(5,8),'max_depth': range(7,10), 'bootstrap': [True,False]}
sgd_list={'loss':['hinge', 'log', 'modified_huber'], 'penalty':['l2', 'l1']}
params_list={DecisionTreeClassifier():dt_list, 
            KNeighborsClassifier(): knn_list, 
            LogisticRegression(): log_list,
            SVC(): svm_list,
            RandomForestClassifier(): rf_list,
            SGDClassifier(): sgd_list
           }

In [53]:
gridresults=pd.DataFrame(columns=['Algorithm','Accuracy','Hyperparameters'])#results from grid search
algoresults=pd.DataFrame(columns=['Algorithm','Accuracy','Hyperparameters'])#results from applying best estimator to algos

for key, value in params_list.items():
    grid=GridSearchCV(key, value)
    grid.fit(X_train, y_train)
    
    model=grid.best_estimator_
    model.fit(X_train, y_train)
    y_predict=model.predict(X_test)
    
    listofalgos1=[key,round(grid.best_score_, 4),grid.best_params_]
    temp1 = pd.DataFrame([listofalgos1], columns=['Algorithm','Accuracy','Hyperparameters'])
    gridresults=pd.concat([temp1, gridresults], axis=0)
    
    listofalgos2=[key,round(accuracy_score(y_test,y_predict),4),grid.best_params_]
    temp2 = pd.DataFrame([listofalgos2], columns=['Algorithm','Accuracy','Hyperparameters'])
    algoresults=pd.concat([temp2, algoresults], axis=0)
    
    


In [54]:
gridresults.reset_index(inplace=True)

In [55]:
gridresults.drop(columns=['index'], inplace=True)

### Table based on Grid Results

In [56]:
gridresults.sort_values(by=['Accuracy'], ascending=False)

,Algorithm,Accuracy,Hyperparameters
1,RandomForestClassifier(),0.8084,"{'bootstrap': True, 'max_depth': 7, 'max_leaf_..."
2,SVC(),0.8048,"{'gamma': 'scale', 'kernel': 'linear'}"
3,LogisticRegression(),0.8048,"{'penalty': 'l1', 'solver': 'liblinear'}"
0,SGDClassifier(),0.7993,"{'loss': 'log', 'penalty': 'l2'}"
4,KNeighborsClassifier(),0.7269,"{'algorithm': 'auto', 'n_neighbors': 3}"
5,DecisionTreeClassifier(),0.7179,"{'criterion': 'entropy', 'max_features': 'sqrt..."


In [57]:
gridresults.nlargest(1,columns='Accuracy') 

,Algorithm,Accuracy,Hyperparameters
1,RandomForestClassifier(),0.8084,"{'bootstrap': True, 'max_depth': 7, 'max_leaf_..."


### Table based on Algorithms with Best hyperparameters 

In [58]:
algoresults.reset_index(inplace=True)

In [59]:
algoresults.drop(columns=['index'], inplace=True)

In [60]:
algoresults.sort_values(by=['Accuracy'], ascending=False)

,Algorithm,Accuracy,Hyperparameters
1,RandomForestClassifier(),0.8525,"{'bootstrap': True, 'max_depth': 7, 'max_leaf_..."
2,SVC(),0.8525,"{'gamma': 'scale', 'kernel': 'linear'}"
3,LogisticRegression(),0.8525,"{'penalty': 'l1', 'solver': 'liblinear'}"
0,SGDClassifier(),0.7869,"{'loss': 'log', 'penalty': 'l2'}"
4,KNeighborsClassifier(),0.7869,"{'algorithm': 'auto', 'n_neighbors': 3}"
5,DecisionTreeClassifier(),0.7705,"{'criterion': 'entropy', 'max_features': 'sqrt..."


In [61]:
algoresults.nlargest(1,columns='Accuracy')

,Algorithm,Accuracy,Hyperparameters
1,RandomForestClassifier(),0.8525,"{'bootstrap': True, 'max_depth': 7, 'max_leaf_..."


In [ ]:
#The end